In [1]:
!pip freeze | grep scikit-leaprn

scikit-learn==1.2.2


In [9]:
import pickle
import pandas as pd
import numpy as np

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [17]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [18]:
df = read_data('./yellow_tripdata_2022-03.parquet')

In [19]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [20]:
y_pred

array([14.28747046, 10.04519858, 28.39097943, ..., 11.63484253,
       11.35071858, 20.94646721])

In [10]:
np.std(y_pred)

5.28140357655334

In [13]:
year = 2022
month = 3
taxi_type="yellow"
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predictions'] = y_pred
output_file = f'./homework.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [21]:
np.mean(y_pred)

12.758556818790902